In [ ]:
import dash
from dash import dcc
from dash import html, ctx
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


  # Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

  # Initialize the Dash app
app = dash.Dash(__name__)

  # Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"

  # Create the dropdown menu options
dropdown_options = [
      {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
      {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
  ]
  # List of years
year_list = [i for i in range(1980, 2024, 1)]

  # Create the layout of the app
app.layout = html.Div([
      html.H1("Automobile Statistics Dashboard"),
      html.Div([
          html.Label("Select Statistics:"),
          dcc.Dropdown(
              id='dropdown-statistics',
              options=dropdown_options,
              value='Yearly Statistics'
          )
      ]),
      html.Div(dcc.Dropdown(
              id='select-year',
              options=[{'label': i, 'value': i} for i in year_list],
              value='Select-year'
          )),
      
      html.Div([
      html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
      ])
  ])

@app.callback(
      Output('select-year', 'disabled'),
      Input('dropdown-statistics','value'))


def update_input_container(selected_statistics):
    if selected_statistics =='Yearly Statistics':
          return False
    else:
          return True


  # Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
    Input(component_id='drop-down-statistics', component_property='value') ])

def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

          #Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
              figure=px.line(yearly_rec,
                  x='year',
                  y='Automobile_Sales',
                  title="Average Automobile Sales fluctuation over Recession Period"))

          #Plot 2 Calculate the average number of vehicles sold by vehicle type
        average_sales = recession_data.groupby('Vehilce_Type')['Automobiles_Sales'].mean().reset_index()
        R_chart2  = dcc.Graph(
              figure=px.bar(average_sales,
                  x='Vehicle',y='Automobile Sales',
                  title= "Average Vehicle Sales for Each Vehicle Category"))

          # Plot pie chart for total expenditure share by vehicle type during recessions
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
              figure=px.pie(exp_rec,
                values='Advertising_Expenditure',
                  names='Vehilce_Type',
                  title="Total Advertising Expenditure Share of Vehicle Type During Recession"
                  )
          )

          # Plot bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
              figure=px.bar(
                  unemp_data,
                  x='unemployment_rate',
                  y='count',
                  color='Blue',
                  labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                  title="Effect of Unemployment Rate on Sales of various Vehicle Types")
          )
        return [
              html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
              html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})
              ]
    elif (input_year and selected_statistics=='Yearly Statistics') :
        yearly_data = data[data['Year'] == input_year]

              #Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        print(yas)
        Y_chart1 = dcc.Graph(figure=px.line(yas,
                                              x='Year',
                                              y='Automobile Sales',title="Yearly Average Automobile Sales"))

              # Total Monthly Automobile sales using line chart.
        Y_chart2 = dcc.Graph(figure=px.line(yearly_data,x='Months',
                      y='Automobiles Sales',title="Monthly Automobile Sales for the year {}".format(input_year)))

              # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=data.groupby('Year')['Automobile_Sales].mean().reset_index()
        print(avr_vdata)
        Y_chart3 = dcc.Graph( figure=px.bar(avr_vdata,
                                              x='Months',
                                              y='Automobile Sales',
                                              title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

              # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehilce_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data,
                                            values='Advertising_Expenditure',
                                            names='Vehilce Type',
                                            title='Total Advertising Expenditure for Vehicle Type in the year {}'.format(input_year)
                      )
              )

        return [
                  html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),
                  html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                  ]

    else:
        return None

  # Run the Dash app
if __name__ == '__main__':
      app.run_server(debug=True)